# Matching two list of author's name for publication
One list of author's name with email needs to be updated with current list of authors on our recent publication. The list needs to be cleaned and linked.

In [1]:
import pandas as pd

In [2]:
curr_authors = 'Joshua T. Vogelstein, Timothy Verstynen, Konrad P. Kording, Leyla Isik, John W. Krakauer, Ralph Etienne-Cummings, Elizabeth L. Ogburn, Carey E. Priebe, Randal Burns, Kwame Kutten, James J. Knierim, James B. Potash, Thomas Hartung, Lena Smirnova, Paul Worley, Alena Savonenko, Ian Phillips, Michael I. Miller, Rene Vidal, Jeremias Sulam, Adam Charles, Noah J. Cowan, Maxim Bichuch, Archana Venkataraman, Chen Li, Nitish Thakor, Justus M Kebschull, Marilyn Albert, Jinchong Xu, Marshall Hussain Shuler, Brian Caffo, Tilak Ratnanather, Ali Geisa, Seung-Eon Roh, Eva Yezerets, Meghana Madhyastha, Javier J. How, Tyler M. Tomita, Jayanta Dey, Ningyuan (Teresa)Huang, Jong M. Shin, Kaleab Alemayehu Kinfu, Pratik Chaudhari, Ben Baker, Anna Schapiro, Dinesh Jayaraman, Eric Eaton, Michael Platt, Lyle Ungar, Leila Wehbe, Adam Kepecs, Amy Christensen, Onyema Osuagwu, Bing Brunton, Brett Mensh, Alysson R. Muotri, Gabriel Silva, Francesca Puppo, Florian Engert, Elizabeth Hillman, Julia Brown, Chris White, Weiwei Yang'

#### Load previous author list which is now outdated

In [3]:
df = pd.read_excel('email-response-list.xlsx')

#### Redact email for privacy

In [4]:
def redact(x):
    if isinstance(x, str):
        s = x.find('<')
        e = x.find('@')
        return x.replace(x[s+1:e], 'XXX')
    else:
        return ''

In [5]:
df.iloc[:,0] = df.apply(lambda x: redact(x.iloc[0]), axis=1)

#### preprocess before probablistic linking

In [6]:
newdf = df.iloc[:,:2].dropna()
newdf.head()

,JOVO EMAIL LIST,WHITE PAPER AUTHOR LIST
0,"Adam Charles <XXX@jhu.edu>,",Adam Charles
1,"Adam Kepecs <XXX@gmail.com>,",Adam Kepecs
2,"Alena Savonenko <XXX@jhmi.edu>,",Alena Savonenko
3,"Ali Geisa <XXX@gmail.com>,",Ali Geisa
4,"Amy Christensen <XXX@gmail.com>,",Amy Christensen


In [7]:
newlist = df.iloc[:,0].tolist()
newlist2 = []

for i in newlist:
    if isinstance(i, str):
        tmp = i.split(' <')
        if len(tmp) == 1:
            continue

        tmp[1] = tmp[1].replace('>,','')
        newlist2.append(tmp)

In [8]:
df_out = pd.DataFrame(newlist2, columns=['Name', 'Email'])
df_out.shape, df_out.head()

((56, 2),
               Name          Email
 0     Adam Charles    XXX@jhu.edu
 1      Adam Kepecs  XXX@gmail.com
 2  Alena Savonenko   XXX@jhmi.edu
 3        Ali Geisa  XXX@gmail.com
 4  Amy Christensen  XXX@gmail.com)

In [9]:
# df_out.to_excel('current_author_list.xlsx', index=False)

#### Get current list of authors in a pandas dataframe

In [10]:
curr_authors_list = [i.strip() for i in curr_authors.split(',')]
curr_authors_list[:10], len(curr_authors_list)

(['Joshua T. Vogelstein',
  'Timothy Verstynen',
  'Konrad P. Kording',
  'Leyla Isik',
  'John W. Krakauer',
  'Ralph Etienne-Cummings',
  'Elizabeth L. Ogburn',
  'Carey E. Priebe',
  'Randal Burns',
  'Kwame Kutten'],
 63)

In [11]:
len(curr_authors_list), curr_authors_list[:10]

(63,
 ['Joshua T. Vogelstein',
  'Timothy Verstynen',
  'Konrad P. Kording',
  'Leyla Isik',
  'John W. Krakauer',
  'Ralph Etienne-Cummings',
  'Elizabeth L. Ogburn',
  'Carey E. Priebe',
  'Randal Burns',
  'Kwame Kutten'])

In [12]:
curr_df = pd.DataFrame(curr_authors_list,columns=['Name'])
curr_df.head()

,Name
0,Joshua T. Vogelstein
1,Timothy Verstynen
2,Konrad P. Kording
3,Leyla Isik
4,John W. Krakauer


#### Join old author list with new author list
Old author list has a column for email whereas the new author list only has the name of authors

In [13]:
import fuzzymatcher

In [14]:
curr_df.head(5), df_out.head(5)

(                   Name
 0  Joshua T. Vogelstein
 1     Timothy Verstynen
 2     Konrad P. Kording
 3            Leyla Isik
 4      John W. Krakauer,
               Name          Email
 0     Adam Charles    XXX@jhu.edu
 1      Adam Kepecs  XXX@gmail.com
 2  Alena Savonenko   XXX@jhmi.edu
 3        Ali Geisa  XXX@gmail.com
 4  Amy Christensen  XXX@gmail.com)

In [15]:
match_result = fuzzymatcher.fuzzy_left_join(
    df_left=curr_df, df_right=df_out, 
    left_on='Name', right_on='Name', 
    left_id_col='Name', right_id_col='Name')

match_result.shape

(63, 6)

In [16]:
match_result.head()

,best_match_score,__id_left,__id_right,Name_left,Name_right,Email
0,NaN,Joshua T. Vogelstein,None,Joshua T. Vogelstein,NaN,NaN
1,0.341898,Timothy Verstynen,Timothy Verstynen,Timothy Verstynen,Timothy Verstynen,XXX@gmail.com
2,0.139637,Konrad P. Kording,Konrad Koerding,Konrad P. Kording,Konrad Koerding,XXX@gmail.com
3,0.270464,Leyla Isik,Leyla Isik,Leyla Isik,Leyla Isik,XXX@jhu.edu
4,0.263982,John W. Krakauer,John Krakauer,John W. Krakauer,John Krakauer,XXX@gmail.com


In [17]:
df_out = match_result[['Name_left', 'Email']]
df_out.columns=['Name','Email']
df_out.head()

,Name,Email
0,Joshua T. Vogelstein,NaN
1,Timothy Verstynen,XXX@gmail.com
2,Konrad P. Kording,XXX@gmail.com
3,Leyla Isik,XXX@jhu.edu
4,John W. Krakauer,XXX@gmail.com


In [18]:
# df_out.to_excel('final_output_email_list.xlsx', index=False)

#### Manual entry of missing emails

In [19]:
df_linear = pd.read_excel('final_output_email_list_post_process.xlsx')

#### Linearize email for convenience

In [20]:
linear_txt = '; '.join(df_linear['Email'])

In [21]:
# output hidden for privacy
# linear_txt

In [22]:
# with open('for_email.txt', 'w', encoding='utf-8') as f:
#     f.write(linear_txt)

#### End of Notebook